# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import nltk
nltk.download('rslp')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [2]:
stemmer = nltk.stem.RSLPStemmer()

In [3]:
def limpa_tweet(tweet):
    tweet = tweet.replace("@", " ")
    tweet = tweet.replace(" por ", " ")
    tweet = tweet.replace(" do ", " ")
    tweet = tweet.replace(" da ", " ")
    tweet = tweet.replace(" de ", " ")
    tweet = tweet.replace(" à ", " ")
    tweet = tweet.replace(" aos ", " ")
    tweet = tweet.replace(" o ", " ")
    tweet = tweet.replace("/", " ")
    tweet = tweet.replace(" a ", " ")
    tweet = tweet.replace(" e ", " ")
    tweet = tweet.replace("!", " ")
    tweet = tweet.replace("?", " ")
    tweet = tweet.replace("(", " ")
    tweet = tweet.replace(")", " ")

    
    tweet = tweet.lower()
    tweet = tweet.split()
    
    tweets_stemizados = []
    for palavra in tweet:
        tweets_stemizados.append(stemmer.stem(palavra))

    return tweets_stemizados

In [4]:
dados = pd.read_excel('tweets_certos_2.0.xlsx')

In [5]:
total_0 = dados.avaliacao[dados.avaliacao == 0].count()
total_1 = dados.avaliacao[dados.avaliacao == 1].count()
total_2 = dados.avaliacao[dados.avaliacao == 2].count()

total = total_0 + total_1 + total_2

In [6]:
p_0 = total_0 / total
p_1 = total_1 / total
p_2 = total_2 / total

print("Probabilidade de Negativo: {:.2f}%".format(p_0*100))
print("Probabilidade de Irrelevante: {:.2f}%".format(p_1*100))
print("Probabilidade de Positivo: {:.2f}%".format(p_2*100))

Probabilidade de Negativo: 52.65%
Probabilidade de Irrelevante: 31.79%
Probabilidade de Positivo: 15.56%


In [7]:
palavras_boas = []
palavras_ruins = []
palavras_neutras = []

frequencia_boas = {}
frequencia_ruins = {}
frequencia_neutras = {}

n_boas = 0
n_ruins = 0
n_neutras = 0

i = 0
for avaliacao in dados['avaliacao']:
    tweet_limpo = limpa_tweet(dados['Treinamento'][i])
    if avaliacao == 0:
        for palavra in tweet_limpo:
            if palavra not in palavras_ruins:
                palavras_ruins.append(palavra)
                frequencia_ruins[palavra] = 1
            else:
                frequencia_ruins[palavra] += 1
            n_ruins += 1
            
    if avaliacao == 1:
        for palavra in tweet_limpo:
            if palavra not in palavras_neutras:
                palavras_neutras.append(palavra)
                frequencia_neutras[palavra] = 1
            else:
                frequencia_neutras[palavra] += 1
            n_neutras += 1
            
    if avaliacao == 2:
        for palavra in tweet_limpo:
            if palavra not in palavras_boas:
                palavras_boas.append(palavra)
                frequencia_boas[palavra] = 1
            else:
                frequencia_boas[palavra] += 1
            n_boas += 1
            
    i += 1
n_total = n_boas + n_neutras + n_ruins


In [8]:
def calc_prob(tweet, dic_palavras, p):
    prob = p
    tweet_limpo = limpa_tweet(tweet)
    for palavra in tweet_limpo:
        ocorrencia = 1
        if palavra in dic_palavras:
            ocorrencia += dic_palavras[palavra]
        prob *= (ocorrencia/n_total)
    return prob

In [9]:
def comparador_prob(prob_0, prob_1, prob_2):
    if prob_0 > prob_1 and prob_0 > prob_2:
        return 0
    if prob_1 > prob_0 and prob_1 > prob_2:
        return 1
    return 2

In [10]:
dados_teste = pd.read_excel("tweets_certos_2.0.xlsx", sheet_name=1)


In [11]:
coluna_previsao = []
for tweet in dados_teste['Teste']:
    prob_0 = calc_prob(tweet, frequencia_ruins, p_0)
    prob_1 = calc_prob(tweet, frequencia_neutras, p_1)
    prob_2 = calc_prob(tweet, frequencia_boas, p_2)
    coluna_previsao.append(comparador_prob(prob_0, prob_1, prob_2))
    
dados_teste['Previsão'] = coluna_previsao

In [12]:
acertos = 0
i = 0
for avaliacao in dados_teste['avaliacao']:
    if avaliacao == dados_teste['Previsão'][i]:
        acertos += 1
    i += 1
    
print("Porcentagem de Acerto: {:.2f}%".format(100*acertos/len(dados_teste)))

Porcentagem de Acerto: 74.50%


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
